In [1]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from esios import *

Create dictionary with Indicator Name and Number of Indicator

In [2]:
indicatorsDict = {
                  'Previsión diaria de la demanda eléctrica peninsular' : 460,
                  'Precio medio horario componente mercado diario' : 805,
                  'Generación T.Real nuclear' : 549,
                  'Generación T.Real hidráulica' : 546,
                  'Generación T.Real eólica' : 551,
                  'Generación T.Real Solar térmica' : 1294,
                  'Generación T.Real Solar fotovoltaica' : 1295
                 }

indicatorsItems = indicatorsDict.items()   # To iterate over name and number at the same time

Download indicators data from 01/01/2016 to 01/06/2020 and save it into `".csv"` files.

In [3]:
start_date = "2016-01-01"
end_date = "2020-05-01"
start_ = start_date + 'T00:00:00'
end_ = end_date + 'T23:50:00'
token = '6cc21e0b60e9931e7522a6ce72a1a09f3a6fadc6f08b142f956db142c6858bc2'    # Introduce ESIOS token
esios = ESIOS(token)
country = 'Spain' #Spain, France or Portugal are the options

for indicatorName, indicatorValue in indicatorsItems:
    print ('Start Date: ' + start_date)
    print ('End Date: ' + end_date)
    indicators_ = list()
    indicators_.append(indicatorValue)
    dfmul , df_list, names = esios.get_multiple_series(indicators_, start_, end_, country)
    df = dfmul[names]
    df = df.reset_index()
    df.columns = ['Date', indicatorName]
    df['Date'] = df['Date'].str.replace('.', ' ')
    df['Date'] = df['Date'].str.split().str[0]
    df['Date'] = df['Date'].str.replace('T', ' ')
    # Export to .csv file
    try:
        os.stat("Files/")
    except:
        os.mkdir("Files/")
    df.to_csv(path_or_buf= 'Files/' + str(indicatorValue) + '.csv', sep='^', index=False)
    print('Generated:' + str(indicatorValue))

Analyzing indicators...
Start Date: 2016-01-01
End Date: 2016-05-01
Generated:460


Read `'.csv'` files generated in different DataFrames.

In [ ]:
with open('Files/460.csv', 'r') as fichero:     
    dfDemand = pd.read_csv(fichero, sep = '^', dtype='object')
    dfDemand['Date']=pd.to_datetime(dfDemand['Date'])
    
with open('Files/1294.csv', 'r') as fichero:     
    dfSolarT = pd.read_csv(fichero, sep = '^', dtype='object')
    dfSolarT['Date']=pd.to_datetime(dfSolarT['Date'])
    
with open('Files/1295.csv', 'r') as fichero:     
    dfSolar = pd.read_csv(fichero, sep = '^', dtype='object')
    dfSolar['Date']=pd.to_datetime(dfSolar['Date']) 
    
with open('Files/551.csv', 'r') as fichero:     
    dfEolic = pd.read_csv(fichero, sep = '^', dtype='object')
    dfEolic['Date']=pd.to_datetime(dfEolic['Date'])
    
with open('Files/805.csv', 'r') as fichero:     
    dfPrice = pd.read_csv(fichero, sep = '^', dtype='object')
    dfPrice['Date']=pd.to_datetime(dfPrice['Date']) 
        
with open('Files/546.csv', 'r') as fichero:     
    dfHidro = pd.read_csv(fichero, sep = '^', dtype='object')
    dfHidro['Date']=pd.to_datetime(dfHidro['Date']) 
        
with open('Files/549.csv', 'r') as fichero:     
    dfNuclear = pd.read_csv(fichero, sep = '^', dtype='object')
    dfNuclear['Date']=pd.to_datetime(dfNuclear['Date']) 

In [ ]:
dfDemand.sample(5)

In [ ]:
dfSolar.sample(5)

In [ ]:
dfEolic.sample(5)

In [ ]:
dfPrice.sample(5)

In [ ]:
dfNuclear.sample(5)